In [0]:
CREATE OR REFRESH STREAMING TABLE cdc_feed
COMMENT "CDC feed for customers"
AS SELECT * FROM STREAM(cdc_demo.cdc_feed_table);

CREATE OR REFRESH STREAMING TABLE customer_table
COMMENT "Target table for CDC data"
TBLPROPERTIES ("quality" = "silver");

In [0]:
APPLY CHANGES INTO LIVE.customer_table
FROM STREAM(LIVE.cdc_feed)
KEYS (customer_id)
APPLY AS DELETE WHEN operation = "DELETE"
SEQUENCE BY version
COLUMNS * EXCEPT (operation, version);

In [0]:
CREATE OR REPLACE MATERIALIZED VIEW customer_table_mv 
AS SELECT customer_id, name, email
FROM LIVE.customer_table;